# LangChain 101

In this tutorial, we will learn LangChain by building a chatbot. For our LangChain chains, let's focus 3 main components *(at least to start with)*: LLMs, Prompts, and Output Parsers.

## LangSmith Setup

*(Optional)* To use LangSmith to trace our chains we want head over to the [LangSmith Settings Page](https://smith.langchain.com/settings), create a new API and do the following to set the enviroment variables:

```python
os.environ["LANGCHAIN_API_KEY"] = <your_api_key>
os.environ["LANGCHAIN_TRACING_V2"] = 'true'
os.environ["LANGCHAIN_PROJECT"] = <your_project_name>
```

Now the traces will be available at https://smith.langchain.com/projects

## LLM

We can use any LLM as the backend since LangChain supports a variety of LLMs, view the full list [here](https://python.langchain.com/v0.2/docs/integrations/chat/). 

We will use **Groq** as  it is free. Create a free API key from https://console.groq.com/keys fro Groq and updated the enviroment variable.

```python
os.environ["GROQ_API_KEY"] = <your_api_key>
```

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## Prompt

Prompt is the first step in the chain. It takes in a dictionary of parameters and returns a string. In this case, we are using a `ChatPromptTemplate` to create a custom prompt template. The `ChatPromptTemplate` takes in a list of tuples, where the first element is the role of the message and the second element is the content of the message. The role can be either `system`, `user`, `assistant`, or `placeholder`. The content of the message can be a string or a variable name that will be replaced with the value of the variable when the prompt is generated.

## Creating a Chain (we are skipping a step)

Now we can create a chain that will take the prompt and the llm and output the response. This is the most basic chain since we have a variable in the prompt that will be filled in by the user and then that is passed to the llm. In Langchain, every chain and llm has a function called `invoke` that takes in a dictionary of variables and returns a response. We make the chains using LCEL which is LangChain Expression Language.

## Output Parsers

As you can see, the output is a class called `AIMessage` that has content, response_metadata and id. Langchain has **Output Parsers** that can.... *you guessed it*, parse the output. There are many outputparsers like JSONOutputParser, PydanticOutputParser, etc. In this case, we are using `StrOutputParser` which parses the output as a string.

<img src="images/AnalyticsVidhya.png" width="700" height="292" />

## Adding Context

In this tutorial, we will add context to our chain. We will have a history of messages that we can use to add context to our chain for our chatbot. We will use `ChatMessageHistory` to store our messages and `RunnableWithMessageHistory` to run our chain with the history. This will basically create a list of messages for our placeholder.

Let's see how this works.

Let's create a prompt template that can hold the chat history.

We can do this manually but it's tedious. Instead, we can use the `RunnableWithMessageHistory` class to do this automatically. 

This also has invoke, although we need to give it a session id so that it can remember the history.

You can see that the messages are stored here automatically.

## RAG

The process of bringing the appropriate information and inserting it into the model prompt is known as **Retrieval Augmented Generation (RAG)**. Retrival Augmented Generation (RAG) is a technique that uses a large language model to generate a response based on a user's input and augmenting LLM knowledge with additional data.

The RAG process involves the following steps:

1. Load the documents; it could be urls, files, etc. See full list of supported loaders [here](https://python.langchain.com/v0.2/docs/integrations/document_loaders/).
2. Split the documents into chunks.
3. Embed the chunks texts into vectors.
4. Store it in a vector database.
5. Query and retrieve the similar documents from the vector database.

Then use it to generate the response.




<img src="images/RAG1.jpg" width="666.67" height="230.67" />

[RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/docs/how_to/recursive_text_splitter/), will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

We can use the `as_retriever()` to create a retriever with your vectorstore. Learn more about it [here](https://python.langchain.com/v0.2/docs/how_to/vectorstore_retriever/)

Use the Retrived documents to generate better responses from the LLM.


<img src="images/RAG2.png" width="506.4" height="259.8"/>

Again, LangChain has two functions that implement the above LCEL:

- `create_stuff_documents_chain` will "stuff" the retrived documents into the prompt.
- `create_retrieval_chain` adds the retriever to get the documents that will be added and propagates the retrieved context through the chain.

Better yet, this one also returns the context of the answer which can be handy. That is basic RAG! 😎

## Conversational Retrieval Augmented Generation

Now we will combine what we learned with chat history and make a Conversational RAG. To do this, we will create a subchain for our retriver, where our input will include the history of the conversation if needed. Before we were just passing the input to the retriever.

Basically, we will contextualize out input with the chat history before passing it to the retriever.


> Note that we leverage a helper function `create_history_aware_retriever` for this step, which manages the case where chat_history is empty, and otherwise applies `prompt | llm | StrOutputParser() | retriever` in sequence.

<img src="images/conversational_retrieval_chain.png" width="792.5" height="371.5"/>

Now we updated the main prompt to include the chat history and create the RAG chain

Everything is done, now we put this chain into the 

Resources/References:

- https://python.langchain.com/v0.2/docs/how_to/message_history/
- https://python.langchain.com/v0.2/docs/tutorials/rag/
- https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/

Contribute on github by making the streamlit code better, or your version of app using different document loaders instead of urls and we can add it!


Next time, we will learn agents and also some langgraph.